In [1]:
import json
import pandas as pd
import numpy as np

# Paths to JSON and CSV
STRUCT_JSON = "qmof_structure_data.json"  # atomic-level DFT site data
MAIN_CSV    = "qmof_joined_with_geometry.csv"

# Load main dataset
df_main = pd.read_csv(MAIN_CSV, low_memory=False)

# Initialize storage for aggregated descriptors
agg_data = []

# Load and aggregate JSON
with open(STRUCT_JSON, 'r') as f:
    data = json.load(f)  # List of dicts, one per qmof_id

    for entry in data:
        qmof_id = entry["qmof_id"]
        sites    = entry.get("structure", {}).get("sites", [])
        # Collect per-atom DFT properties
        # Keys under each site["properties"] we want to aggregate:
        props = [
            "pbe_ddec_charge",
            "pbe_cm5_charge",
            "pbe_bader_charge",
            "pbe_ddec_spin_density",
            "pbe_bader_spin_density",
            "pbe_magmom"
        ]
        # Build dict of lists
        values = {p: [] for p in props}
        for site in sites:
            pr = site.get("properties", {})
            for p in props:
                if p in pr:
                    values[p].append(pr[p])

        # Compute aggregations for each prop
        row = {"qmof_id": qmof_id}
        for p, lst in values.items():
            arr = np.array(lst, dtype=float)
            if arr.size > 0:
                row[f"{p}_mean"]    = np.mean(arr)
                row[f"{p}_median"]  = np.median(arr)
                row[f"{p}_std"]     = np.std(arr)
                row[f"{p}_q25"]     = np.percentile(arr, 25)
                row[f"{p}_q75"]     = np.percentile(arr, 75)
                row[f"{p}_max"]     = np.max(arr)
                row[f"{p}_min"]     = np.min(arr)
            else:
                # No data for this MOF
                for suffix in ["mean", "median", "std", "q25", "q75", "max", "min"]:
                    row[f"{p}_{suffix}"] = np.nan
        agg_data.append(row)

# Convert to DataFrame
df_agg = pd.DataFrame(agg_data)

# Merge with main DataFrame on qmof_id
df_merged = df_main.merge(df_agg, on="qmof_id", how="left")

# Save to new CSV
df_merged.to_csv("qmof_with_dft_aggregates.csv", index=False)

# Show resulting columns
print("New columns added:")
print([c for c in df_agg.columns if c != "qmof_id"])


New columns added:
['pbe_ddec_charge_mean', 'pbe_ddec_charge_median', 'pbe_ddec_charge_std', 'pbe_ddec_charge_q25', 'pbe_ddec_charge_q75', 'pbe_ddec_charge_max', 'pbe_ddec_charge_min', 'pbe_cm5_charge_mean', 'pbe_cm5_charge_median', 'pbe_cm5_charge_std', 'pbe_cm5_charge_q25', 'pbe_cm5_charge_q75', 'pbe_cm5_charge_max', 'pbe_cm5_charge_min', 'pbe_bader_charge_mean', 'pbe_bader_charge_median', 'pbe_bader_charge_std', 'pbe_bader_charge_q25', 'pbe_bader_charge_q75', 'pbe_bader_charge_max', 'pbe_bader_charge_min', 'pbe_ddec_spin_density_mean', 'pbe_ddec_spin_density_median', 'pbe_ddec_spin_density_std', 'pbe_ddec_spin_density_q25', 'pbe_ddec_spin_density_q75', 'pbe_ddec_spin_density_max', 'pbe_ddec_spin_density_min', 'pbe_bader_spin_density_mean', 'pbe_bader_spin_density_median', 'pbe_bader_spin_density_std', 'pbe_bader_spin_density_q25', 'pbe_bader_spin_density_q75', 'pbe_bader_spin_density_max', 'pbe_bader_spin_density_min', 'pbe_magmom_mean', 'pbe_magmom_median', 'pbe_magmom_std', 'pbe_m